In [ ]:
### 🛑 Import des bibliothèques
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy import fft
import time

# L'algorithme *Fast Fourier Transform* (FFT)

En calculant les coefficient fréquentiels d'un signal de manière optimisée, l'algorithme *Fast Fourier Transform* (FFT) réduit la complexité de calcul de la TFD de $\mathcal{O}(N^2)$ à $\mathcal{O}(N \log N)$, ce qui permet de traiter des signaux de grande taille de manière efficace.

L'algorithme FFT est souvent associé à la version de Cooley et Tukey, proposée en 1965, qui repose sur une factorisation récursive du signal en deux sous-signaux, l'un correspondant aux indices pairs et l'autre aux indices impairs. Cependant, il est intéressant de noter que cette méthode était déjà connue bien avant, notamment par Carl Friedrich Gauss dès 1805, qui utilisait une approche similaire pour des calculs astronomiques (si vous voulez en lire un peu plus à propos de l'histoire de la FFT, c'est [par ici](https://www.cis.rit.edu/class/simg716/Gauss_History_FFT.pdf)).

La clé du succès de la FFT réside dans l'exploitation des symétries de la matrice de Fourier. En décomposant intelligemment le signal en parties plus petites (diviser pour régner), et en réutilisant les calculs effectués pour les sous-parties du signal, la FFT parvient à réduire le nombre d'opérations nécessaires.

## Principe de récursion de la FFT

L'algorithme FFT repose sur un principe de **diviser pour régner**, qui s'applique efficacement lorsque la taille du signal est une puissance de 2. La FFT divise le problème de calcul de la TFD pour un signal de taille $N$ en deux sous-problèmes de taille $N/2$, ce qui permet au final de réduire la complexité de $\mathcal{O}(N^2)$ à $\mathcal{O}(N \log N)$, en exploitant les symétries naturelles de la matrice de Fourier.

En particulier, en repartant de la formule du $k$ième coefficient de la TFD $X[k]$ :

$$X[k] = \displaystyle \sum_{n=0}^{N-1} x[n] e^{-i 2 \pi \large\frac{nk}{N}} \quad k = 0,\dots,N-1 \quad (1)$$

On a 
$$X[k+N] = \displaystyle \sum_{n=0}^{N-1} x[n] e^{-i 2 \pi \large\frac{n(k+N)}{N}},$$
que l'on peut développer en
$$X[k+N]  = \sum_{n=0}^{N-1} x[n] e^{-i 2 \pi \large\frac{nk}{N}}e^{-i 2 \pi \large\frac{nN}{N}} = \sum_{n=0}^{N-1} x[n] e^{-i 2 \pi \large\frac{nk}{N}}e^{-i 2 \pi n}.$$
Et puisque $e^{-i 2 \pi n} = 1 \ \ \forall n\in \mathbb{Z}$, on a au final
$$X[k+N] = \sum_{n=0}^{N-1} x[n] e^{-i 2 \pi \large\frac{nk}{N}} = X[k]$$
qui s'étend naturellement à $X[k+pN] = X[k]$ pour n'importe quel entier $p \in \mathbb{Z}$

Autrement dit, le coefficient $X[k]$ est $N-$périodique.

OK, très bien, mais à quoi ça sert en pratique ? Avant de voir en quoi cette propriété permet d'accélérer le calcul des coefficients de la TFD, vérifions rapidement qu'elle est bien vraie expérimentalement.

In [ ]:
# 🛑 Définition d'un signal x aléatoire
N = 16 # peut être n'importe quelle autre valeur (et pas nécéssairement une puissance de 2)
x = np.random.randint(10,size=N)

In [ ]:
# 🛑 Définition du calcul de X[k] via une lambda
X_N = lambda k : np.sum([x[n]*np.exp(-1j*2*np.pi*n*k/N) for n in range(N)])

In [ ]:
# 🛑 Vérification que X[k] = X[k+N]
k = 5 # doit être compris entre 0 et N-1
print(np.isclose(X_N(k),X_N(k+N)))

In [ ]:
# 🛑 D'ailleurs, ça marche pour X[k] = X[k+pN] avec p∈ℤ
p = -3 # n'importe quel entier relatif
print(np.isclose(X_N(k),X_N(k+p*N)))

### Exploitation de la symétrie

**📢 Prenez le temps pour lire et bien comprendre cette partie, elle est cruciale pour l'algorithme FFT 📢**

Repartons (encore) de la définition de $X[k]$ :
$$X[k] = \displaystyle \sum_{n=0}^{N-1} x[n] e^{-i 2 \pi \large\frac{nk}{N}} \quad k = 0,\dots,N-1 \quad (1)$$

Sans perte de généralité, on peut décomposer cette sommation en deux : une portant sur les indices pairs $(n=2m)$, et l'autre sur les indices impairs $(n=2m+1)$:

$$ X[k] = \sum_{\substack{ n = 0 \\[1pt] n = 2m}}^{N-1} x[n] e^{-i 2 \pi \large\frac{k n}{N}} + \sum_{\substack{ n = 0 \\[1pt] n = 2m+1}}^{N-1} x[n] e^{-i 2 \pi \large\frac{k n}{N}}$$

En effectuant le changement de variable $n = 2m$ pour la somme sur les indices pairs, et $n = 2m+1$ pour celle sur les indices impairs, on peut réécrire

$$ X[k] = \sum_{m=0}^{N/2-1} x[2m] e^{-i 2 \pi \large\frac{k (2m)}{N}} + \sum_{m=0}^{N/2-1} x[2m+1] e^{-i 2 \pi \large\frac{k (2m+1)}{N}}$$

* Pour le premier terme $\displaystyle \sum_{m=0}^{N/2-1} x[2m] e^{-i 2 \pi \large\frac{k (2m)}{N}}$, on va "juste" réécrire $e^{-i 2 \pi \large\frac{k (2m)}{N}} = e^{-i 2 \pi \large\frac{k m}{N/2}}$

* Pour le deuxième terme $\displaystyle  \sum_{m=0}^{N/2-1} x[2m+1] e^{-i 2 \pi \large\frac{k (2m+1)}{N}}$, on va procéder de même : $e^{-i 2 \pi \large\frac{k (2m+1)}{N}} = e^{-i 2 \pi \large\frac{k}{N}}e^{-i 2 \pi \large\frac{k (2m)}{N}} = e^{-i 2 \pi \large\frac{k}{N}}e^{-i 2 \pi \large\frac{k m}{N/2}}$<br>
On retrouve bien le même terme $e^{-i 2 \pi \large\frac{k m}{N/2}}$, mais cette fois-ci, il y a un facteur $e^{-i 2 \pi \large\frac{k}{N}}$ qui se factorise devant.

<br>
On arrive à l'expression finale de $X[k]$ :
<div style="border: 1px solid red; padding: 5px; display: inline-block;">
$$ X[k] = \sum_{m=0}^{N/2-1} x[2m] e^{-i 2 \pi \large\frac{k m}{N/2}} + e^{-i 2 \pi \large\frac{k}{N}} \sum_{m=0}^{N/2-1} x[2m+1] e^{-i 2 \pi \large\frac{k m}{N/2}} \quad (2)$$
</div>

Cette formule barbare appelle deux remarques :
1. les termes qui apparaissent dans les deux sommes sont en fait les TFD de deux séquences de taille $\frac{N}{2}$ (le $N$ qui apparaissait dans l'équation $(1)$ a bien été remplacé par $\frac{N}{2}$ dans l'équation $(2)$) :
    - pour la première somme, la séquence ne comprend que les éléments d'indices pairs de $x$ : $ [x[0], x[2], x[4], \ldots] = x_{\text{pair}}$
    - pour la deuxième somme, c'est pareil, mais avec les indices impairs : $[x[1], x[3], x[5], \ldots] = x_{\text{impair}}$
   
   L'équation (2) peut se réécrire $$X[k] = X_{\text{pair}}[k] + e^{-i 2 \pi \large\frac{k}{N}} X_{\text{impair}}[k] \quad (3)$$
    où :
    - $ X_{\text{pair}}[k] $ représente la TFD calculée uniquement avec les échantillons pairs $x_{\text{pair}}$ de $x$.
    - $ X_{\text{impair}}[k] $ représente la TFD calculée uniquement avec les échantillons impairs $x_{\text{impair}}$ de $x$.
    - Le terme $ e^{-i 2 \pi \large\frac{k}{N}} $ est un facteur de phase qui modifie l'influence des échantillons impairs sur le résultat final.
<br>
<br>
2. Pour chaque sous-problème, on a $0 \leq m \leq \frac{N}{2}$, mais $0 \leq k \leq N-1$. La propriété de symétrie permet donc de ne calculer que la moitié des coefficients $X[k]$ (pour $k=0,\dots, \frac{N}{2}-1$), puisque l'autre moitié (pour $k = \frac{N}{2},\dots, N-1$) s'en déduira immédiatement par symétrie.

Au final, les $\mathcal{O}(N^2)$ opérations de la TFD de base se sont transformées en $\mathcal{O}(M^2)$ opérations pour ce premier niveau de récursion, avec $M = \frac{N}{2}$. En appliquant récursivement cettre stratégie, on arrive au final à un coût total de $\mathcal{O}(N \log(N))$ 
(vous pouvez appliquer le [*master theorem*](https://fr.wikipedia.org/wiki/Master_theorem) de votre cours d'algorithmique pour vous en convaincre si besoin).

⚠️ Ces différentes récursions ne peuvent s'appliquer efficacement que si, de base, la longueur totale $N$ du signal $x$ est elle-même une puissance de 2. **On va donc se placer dans ce cas de figure par la suite**.<br>
À noter tout de même que si $N$ n'est pas une puissance de 2, on peut quand même s'en sortir (ne serait-ce qu'en complétant le signal $x$ avec des échantillons nuls pour que sa nouvelle taille soit une puissance de 2).

## Un premier niveau de récursion

Prenons un exemple avec un signal de taille $N = 8$ : $x = [x[0], x[1],\dots,x[7]]$<br>
Pour le premier niveau de récursion, l'algorithme FFT divise ce signal en deux sous-signaux de taille 4 : 
* $x_{\text{pair}}$ qui contient les **échantillons d'indices pairs** de $x$ : $x_{\text{pair}} = [x[0], x[2], x[4], x[6]]$
* $x_{\text{impair}}$ qui contient les **échantillons d'indices impairs** de $x$ : $x_{\text{impair}} = [x[1], x[3], x[5], x[7]]$

La TFD de ces deux sous-signaux $x_\text{pair}$ et $x_{\text{impair}}$ est calculée soit par la méthode naïve, soit (dans le cas de la FFT) avec une nouvelle récursion, résultant en $X_{\text{pair}}$ et $X_{\text{impair}}$. À noter qu'ici, $X_{\text{pair}}$ et $X_{\text{impair}}$ sont aussi de taille $\frac{N}{2} = 4$ (comme $x_\text{pair}$ et $x_{\text{impair}}$).

La TFD $X$ de $x$, de taille $N = 8$, est recomposée en recombinant $X_{\text{pair}}$ et $X_{\text{impair}}$ grâce à l'équation (3) :
* Pour $k=0,\dots, 3 \small=\frac{N}{2}-1$, on a directement $X[k] = X_{\text{pair}}[k] + e^{-i 2 \pi \large\frac{k}{8}} X_{\text{impair}}[k]$ $\qquad\qquad\qquad (4.1)$
* Pour $k=4 = \frac{N}{2},\dots, 7 =N-1$, on a $X[k] = X_{\text{pair}}[k-4] + e^{-i 2 \pi \large\frac{k}{8}} X_{\text{impair}}[k-4]$ $\qquad\qquad\quad (4.2)$<br>
En effet, $$\begin{align}
X_{\text{pair}}[4] = X_{\text{pair}}[0+4] = X_{\text{pair}}[0]\\
X_{\text{pair}}[5] = X_{\text{pair}}[1+4] = X_{\text{pair}}[1]\\
X_{\text{pair}}[6] = X_{\text{pair}}[2+4] = X_{\text{pair}}[2]\\
X_{\text{pair}}[7] = X_{\text{pair}}[3+4] = X_{\text{pair}}[3]
\end{align}$$
en utilisant la propriété de symétrie de $X_{\text{pair}}$ (de taille 4). Idem pour $X_{\text{impair}}$.

Ce niveau de récursion peut s'illustrer par la figure suivante :
<img src="https://vocal.com/wp-content/uploads/2014/03/FFT-1024x502.png" style="width:75%; height:auto;">

<br>
Dans cette figure précédente, $X_{ev}$ et $X_{od}$ correspondent respectivement à $X_\text{pair}$ et $X_\text{impair}$, et $W_8^k = e^{-i 2 \pi \large\frac{k}{8}}$ avec $k=0,\dots,7$

Il est temps de mettre tout ça en pratique ! Dans un premier temps, on va se restreindre au premier niveau de récursion pour un signal de taille $N = 8$, que l'on va redéfinir avec des entrées aléatoires (puisqu'encore une fois, la valeur des entrées de $x$ n'a absolument aucune importance ici).

In [ ]:
# 🛑 Définition d'un signal de taille 8 et d'entrées aléatoires dans [0,10[
x = np.random.randint(10,size=8)
print("Le signal x =",x, "de taille N = %d"%x.size)

Pour calculer la TFD des deux sous-signaux $x_\text{pair}$ et $x_{\text{impair}}$, on va pour le moment réutiliser l'implémentation naïve. Vous pouvez donc soit réimporter ici ce que vous avez fait dans l'[exercice précédent](TP4_AlgorithmeFFT_exo1.ipynb), soit vous servir du code qui vous est donné ici et qui exploite les optimisations fournies par `numpy` (pas de boucles), même si ça ne change pas sa complexité algorithmique)

In [ ]:
# 🛑 Calcul de la TFD naïve, numpy style

# Définition de la matrice de Fourier de taille NxN
def Fourier_matrix(N):
    nn = np.arange(N)
    kk = nn.reshape(N,1)
    F = np.exp(-1j*2*np.pi*nn*kk/N)
    return F

# TFD
def DFT_naive(x):
    F = Fourier_matrix(x.size)
    X = F@x # produit matrice/vecteur avec les optims de numpy
    return X

### 🛠️ 🚧 👷  À vous de jouer !

Définissez les sous-signaux $x_\text{pair}$ et $x_{\text{impair}}$ en ne récupérant que les indices pairs et impairs de $x$, puis calculez leur TFD $X_\text{pair}$ et $X_{\text{impair}}$ par la méthode naïve. Vérifiez que $x_\text{pair}$, $x_{\text{impair}}$, $X_\text{pair}$ et $X_{\text{impair}}$ sont bien tous de taille $\frac{N}{2} = 4$

<u>Python pro tip :</u> Pour récupérer un point sur deux d'un `array` `x` en commençant à l'indice `k`, vous pouvez écrire `x[k::2]`

In [ ]:
x_pair = ??? # FIXME
x_impair = ??? # FIXME

In [ ]:
X_pair = ??? # FIXME
X_impair = ??? # FIXME

In [ ]:
print("Taille de x_pair : %d"%x_pair.size)
print("Taille de x_impair : %d"%x_impair.size)
print("Taille de X_pair : %d"%X_pair.size)
print("Taille de X_impair : %d"%X_impair.size)

### 🛠️ 🚧 👷  À vous de jouer !

Reconstruisez maintenant $X$ grâce à $X_\text{pair}$ et $X_\text{impair}$ en utilisant l'équation $(4.1)$ pour les coefficient $X[k], \ \ k=0,\dots, 3$ et l'équation $(4.2)$ pour les coefficient $X[k], \ \ k=4,\dots, 7$

In [ ]:
def recombine_DFT(X_pair,X_impair):
    N = 2*X_pair.size # puisque X_pair de taille N/2
    X = np.zeros(N).astype('complex') # Initialisation de X de type complexe
    for k in range(int(N/2)): # k=0,...,3 ici → utiliser l'équation (4.1)
        X[k] = ??? # FIXME
    for k in range(int(N/2),N): # k=4,...,7 ici → utiliser l'équation (4.2)
        X[k] = ??? # FIXME  
    return X

### 🛠️ 🚧 👷  À vous de jouer !

Pour finir, vérifiez que ce que vous avez obtenu est bien équivalent au calcul de la TFD naïve sur le signal de taille $N=8$

In [ ]:
X = recombine_DFT(X_pair,X_impair)
np.isclose(X, DFT_naive(x))

### Et ensuite ?

Et ensuite, la récursion continue dans l'algorithme FFT. Chacun de ces sous-signaux de taille 4 est à son tour divisé en deux sous-signaux de taille 2, qui sont à leur tour divisés en deux sous-signaux de taille 1 (et la récursion s'arrête à ce niveau la).<br>
Et à chaque étape, l'algorithme réutilise les résultats des sous-problèmes pour calculer le résultat final de la TFD en combinant les sous-résultats.

## L'algorithme FFT final

Vous avez maintenant toutes les briques nécessaires pour implémenter l'intégralité de l'algorithme FFT : ne reste plus qu'à pousser les récursions jusqu'au bout !

### 🛠️ 🚧 👷  À vous de jouer !

Un dernier petit effort ! Complétez la cellule ci-dessous pour implémenter la récursion dans l'algorithme FFT jusqu'au niveau final, puis vérifiez que vous obtenez bien les mêmes résultats qu'avec la TFD naïve et `sp.fft.fft`

In [ ]:
# Implémentation de l'algorithme FFT avec la récursion complète
def FFT(x):
    if x.size == 1:
        return x # se retourne lui-même dans le cas d'un signal de longeur 1
    else:
        x_pair = ??? # FIXME sous-signal des indices pairs
        X_pair = ??? # FIXME Récursion
        x_impair = ??? # FIXME sous-signal des indices impairs
        X_impair = ??? # FIXME Récursion
        X = ??? # FIXME Recombinaison des résultats du niveau de récursion précédent
        return X        

In [ ]:
np.isclose(FFT(x), DFT_naive(x))

In [ ]:
np.isclose(FFT(x), sp.fft.fft(x))

## Vérification de la complexité en $\mathcal{O}(N \log(N))$ pour l'implémentation FFT

Ne reste plus qu'à vérifier que l'implémentation de l'algorithme FFT tourne bien en $\mathcal{O}(N \log(N))$ (ça serait un peut décevant d'avoir fait tout ça sinon...). On va utiliser exactement la même stratégie que dans l'exercice précédent pour l'implémentation naïve, notre optimisme sur le résultat à venir nous poussant néanmoins à augmenter l'étendue de la plage des tailles $N$, en l'occurence entre $N=2^4$ et $N = 2^{17}$

In [ ]:
# 🛑 liste des tailles N (de la forme 2^p)
list_N = 2**np.arange(4,18)
print("Liste des tailles N pour la FFT :",list_N)

### 🛠️ 🚧 👷  À vous de jouer !

Complétez la cellule suivante pour calculer, pour chaque longueur $N$ dans `list_N`, le temps de calcul de l'algorithme FFT d'un signal aléatoire de longueur $N$. Tracez ensuite le temps de calcul en fonction de $N$. <br>
L'implémentation de la FFT est-elle bien en $\mathcal{O}(N \log(N))$ ?

In [ ]:
t_elapse = []
for N in list_N:
    x = np.random.randint(10,size=???) # FIXME
    t_begin = ??? # FIXME
    X = ??? # FIXME calcul de la transformée de Fourier avec notre implémentation de la FFT
    t_end = ??? # FIXME
    t_elapse.append((t_end-t_begin))

In [ ]:
# 🛑 Temps de calcul de la FFT vs O(Nlog(N))
plt.figure(figsize=(8,5))
idx_norm = np.round(np.log10(list_N[-1]*np.log10(list_N[-1])))
plt.loglog(list_N,list_N*np.log10(list_N)/(10**idx_norm),'k--',label=r'$\mathcal{O}(N \log(N))$')
plt.loglog(list_N,t_elapse,'b',label=r'$t_{FFT}$')
plt.xlim(list_N.min(),list_N.max())
plt.xlabel(r'Taille du signal $N$',fontsize=14)
plt.title(r'Temps de calcul de la FFT',fontsize=14)
plt.legend(loc='best',fontsize=14)
plt.show()

# Bravo ! 👏🎉

Vous en êtes arrivés enfin à bout ! Et c'était un sacré morceau 🍻